In [1]:
import zipfile
import glob
import zarr
from netCDF4 import Dataset
from pathlib import Path

# Convert a netcdf file to zarr

** install pyutils to get the data_read.download function and tracking to get the download and single_zarr functions**

In [13]:
!conda install -y fastparquet python-snappy

Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /Users/phil/mini36:

The following NEW packages will be INSTALLED:

    python-snappy: 0.5.1-py36_0 conda-forge

python-snappy- 100% |################################| Time: 0:00:00 325.62 kB/s


In [3]:
!conda install -y pyutils tracking -c phaustin

Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/phil/mini36:
#
pyutils                   0.1                        py_0    phaustin
tracking                  0.1                        py_0    phaustin


** download the netcdf files and unzip **

In [4]:
from pyutils.data_read import download

In [5]:
the_zip = 'test.zip'
download(the_zip)

trying https://clouds.eos.ubc.ca/~phil/courses/atsc301/downloads/test.zip
writing to: test.zip

test.zip already exists
and is 95492218 bytes
will not overwrite



In [6]:
with zipfile.ZipFile(the_zip,"r") as zip_ref:
    zip_ref.extractall(".")

In [7]:
from tracking.single_zarr import convert_file, dump_meta

In [8]:
nc_files = glob.glob('*.nc')  
for the_file in nc_files:
    nc_path=Path(the_file)
    #
    # make the output directory the same as the input nc name
    # with .zarr instead of .nc
    #
    zarr_dir=nc_path.with_suffix('.zarr')
    convert_file(the_file,zarr_dir)
    #
    # store the metadata from the netcdf file in a nested dictionary
    #
    meta_dict=dump_meta(the_file)
    store = zarr.DirectoryStore(zarr_dir)
    #
    # open the file in append mode and transfer the metadata
    #
    the_group=zarr.hierarchy.open_group(store=store, mode='a',
                                        synchronizer=None, path=None)
    for key,value in meta_dict['global'].items():
        the_group.attrs[key]=value
    del meta_dict['global']
    for var,var_dict in meta_dict.items():
        for key, value in var_dict.items():
            the_group[var].attrs[key]=value
            print(var,list(the_group[var].attrs.keys()))


variable: Times
variable: XLAT
variable: XLONG
variable: LU_INDEX
variable: ZNU
variable: ZNW
variable: ZS
variable: DZS
variable: U
variable: V
variable: W
variable: PH
variable: PHB
variable: T
variable: HFX_FORCE
variable: LH_FORCE
variable: TSK_FORCE
variable: HFX_FORCE_TEND
variable: LH_FORCE_TEND
variable: TSK_FORCE_TEND
variable: MU
variable: MUB
variable: P
variable: PB
variable: P_HYD
variable: Q2
variable: T2
variable: TH2
variable: PSFC
variable: U10
variable: V10
variable: ZETATOP
variable: ITIMESTEP
variable: XTIME
variable: QVAPOR
variable: QCLOUD
variable: QRAIN
variable: QICE
variable: QSNOW
variable: TSLB
variable: SMOIS
variable: SH2O
variable: SMCREL
variable: SFROFF
variable: IVGTYP
variable: ISLTYP
variable: VEGFRA
variable: GRDFLX
variable: ACGRDFLX
variable: ACSNOM
variable: SNOW
variable: SNOWH
variable: SSTSK
variable: COSZEN
variable: LAI
variable: Z0
variable: VAR
variable: MAPFAC_M
variable: MAPFAC_U
variable: MAPFAC_V
variable: MAPFAC_MX
variable: MAPFAC_MY

LU_INDEX ['FieldType', 'MemoryOrder', 'description', 'units']
LU_INDEX ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
LU_INDEX ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
ZNU ['FieldType']
ZNU ['FieldType', 'MemoryOrder']
ZNU ['FieldType', 'MemoryOrder', 'description']
ZNU ['FieldType', 'MemoryOrder', 'description', 'units']
ZNU ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
ZNW ['FieldType']
ZNW ['FieldType', 'MemoryOrder']
ZNW ['FieldType', 'MemoryOrder', 'description']
ZNW ['FieldType', 'MemoryOrder', 'description', 'units']
ZNW ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
ZS ['FieldType']
ZS ['FieldType', 'MemoryOrder']
ZS ['FieldType', 'MemoryOrder', 'description']
ZS ['FieldType', 'MemoryOrder', 'description', 'units']
ZS ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
DZS ['FieldType']
DZS ['FieldType', 'MemoryOrder']
DZS ['FieldType', 'MemoryOrder', 'description']
DZS ['Field

QVAPOR ['FieldType', 'MemoryOrder']
QVAPOR ['FieldType', 'MemoryOrder', 'description']
QVAPOR ['FieldType', 'MemoryOrder', 'description', 'units']
QVAPOR ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
QVAPOR ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
QCLOUD ['FieldType']
QCLOUD ['FieldType', 'MemoryOrder']
QCLOUD ['FieldType', 'MemoryOrder', 'description']
QCLOUD ['FieldType', 'MemoryOrder', 'description', 'units']
QCLOUD ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
QCLOUD ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
QRAIN ['FieldType']
QRAIN ['FieldType', 'MemoryOrder']
QRAIN ['FieldType', 'MemoryOrder', 'description']
QRAIN ['FieldType', 'MemoryOrder', 'description', 'units']
QRAIN ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
QRAIN ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
QICE ['FieldType']
QICE ['FieldType', 'MemoryOrder']

TISO ['FieldType', 'MemoryOrder', 'description', 'units']
TISO ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
TLP_STRAT ['FieldType']
TLP_STRAT ['FieldType', 'MemoryOrder']
TLP_STRAT ['FieldType', 'MemoryOrder', 'description']
TLP_STRAT ['FieldType', 'MemoryOrder', 'description', 'units']
TLP_STRAT ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
P_STRAT ['FieldType']
P_STRAT ['FieldType', 'MemoryOrder']
P_STRAT ['FieldType', 'MemoryOrder', 'description']
P_STRAT ['FieldType', 'MemoryOrder', 'description', 'units']
P_STRAT ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
MAX_MSTFX ['FieldType']
MAX_MSTFX ['FieldType', 'MemoryOrder']
MAX_MSTFX ['FieldType', 'MemoryOrder', 'description']
MAX_MSTFX ['FieldType', 'MemoryOrder', 'description', 'units']
MAX_MSTFX ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
MAX_MSTFY ['FieldType']
MAX_MSTFY ['FieldType', 'MemoryOrder']
MAX_MSTFY ['FieldType', 'MemoryOrder', 'description']
MAX_

LH ['FieldType', 'MemoryOrder', 'description', 'units']
LH ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
LH ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
ACHFX ['FieldType']
ACHFX ['FieldType', 'MemoryOrder']
ACHFX ['FieldType', 'MemoryOrder', 'description']
ACHFX ['FieldType', 'MemoryOrder', 'description', 'units']
ACHFX ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
ACHFX ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
ACLHF ['FieldType']
ACLHF ['FieldType', 'MemoryOrder']
ACLHF ['FieldType', 'MemoryOrder', 'description']
ACLHF ['FieldType', 'MemoryOrder', 'description', 'units']
ACLHF ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
ACLHF ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
PREC_ACC_C ['FieldType']
PREC_ACC_C ['FieldType', 'MemoryOrder']
PREC_ACC_C ['FieldType', 'MemoryOrder', 'description']
PREC_ACC_C ['FieldType', 'MemoryOrder

u_tr_p ['long_name']
u_tr_p ['long_name', 'standard_name']
u_tr_p ['long_name', 'standard_name', 'units']
u_tr_p ['_FillValue', 'long_name', 'standard_name', 'units']
u_tr_p ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
v_tr_p ['long_name']
v_tr_p ['long_name', 'standard_name']
v_tr_p ['long_name', 'standard_name', 'units']
v_tr_p ['_FillValue', 'long_name', 'standard_name', 'units']
v_tr_p ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
w_p ['long_name']
w_p ['long_name', 'standard_name']
w_p ['long_name', 'standard_name', 'units']
w_p ['_FillValue', 'long_name', 'standard_name', 'units']
w_p ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
albedo ['long_name']
albedo ['long_name', 'standard_name']
albedo ['long_name', 'standard_name', 'units']
albedo ['_FillValue', 'long_name', 'standard_name', 'units']
albedo ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
SH ['long_name']
SH ['long_name', 'standard

In [9]:
!ls -ltd *zarr

drwxr-xr-x   58 phil  staff  1972 Jul  4 13:08 wrfout_d03_2016-01-04_09:00:00_post.zarr
drwxr-xr-x  121 phil  staff  4114 Jul  4 13:08 wrfout_d03_2016-01-04_09:00:00.zarr


In [10]:
!du -h wrfout*

148M	wrfout_d03_2016-01-04_09:00:00.nc
108K	wrfout_d03_2016-01-04_09:00:00.zarr/ACGRDFLX
152K	wrfout_d03_2016-01-04_09:00:00.zarr/ACHFX
152K	wrfout_d03_2016-01-04_09:00:00.zarr/ACLHF
 32K	wrfout_d03_2016-01-04_09:00:00.zarr/ACSNOM
 72K	wrfout_d03_2016-01-04_09:00:00.zarr/ALBEDO
 88K	wrfout_d03_2016-01-04_09:00:00.zarr/CLAT
2.2M	wrfout_d03_2016-01-04_09:00:00.zarr/CLDFRA
 96K	wrfout_d03_2016-01-04_09:00:00.zarr/COSALPHA
 88K	wrfout_d03_2016-01-04_09:00:00.zarr/COSZEN
 12K	wrfout_d03_2016-01-04_09:00:00.zarr/DZS
 64K	wrfout_d03_2016-01-04_09:00:00.zarr/EMISS
132K	wrfout_d03_2016-01-04_09:00:00.zarr/GLW
 32K	wrfout_d03_2016-01-04_09:00:00.zarr/GRAUPELNC
108K	wrfout_d03_2016-01-04_09:00:00.zarr/GRDFLX
 32K	wrfout_d03_2016-01-04_09:00:00.zarr/HAILNC
152K	wrfout_d03_2016-01-04_09:00:00.zarr/HFX
 12K	wrfout_d03_2016-01-04_09:00:00.zarr/HFX_FORCE
 12K	wrfout_d03_2016-01-04_09:00:00.zarr/HFX_FORCE_TEND
108K	wrfout_d03_2016-01-04_09:00:00.zarr/HGT
 40K	wrfout_d03_2016-01-04_09:00:00.zarr/ISLTYP


# Convert a csv file to a pandas dataframe

In [11]:
the_csv = '684text.csv'
download(the_csv)

trying https://clouds.eos.ubc.ca/~phil/courses/atsc301/downloads/684text.csv
writing to: 684text.csv

684text.csv already exists
and is 31785 bytes
will not overwrite



## read the csv file into a dataframe named out_df, and write to a .parq file

In [14]:
from fastparquet import write, ParquetFile
import pandas as pd
in_file=Path(the_csv)
out_file=in_file.with_suffix('.parq')
print(f'here is the output name {str(out_file)}')
out_df=pd.read_csv(str(in_file))
write(str(out_file),out_df,object_encoding='utf8',compression='SNAPPY')

here is the output name 684text.parq


RuntimeError: Compression 'SNAPPY' not available.  Options: ['GZIP', 'UNCOMPRESSED']

In [ ]:
out_df

In [ ]:
round_trip_df=ParquetFile(str(out_file)).to_pandas()
print('here is the round trip')
print(round_trip_df.head())